In [1]:
# Import statements
# Declare Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
import os
import pickle
from textwrap import wrap
import operator
import au_census_analysis_functions as cnss_func

# Set a variable for current notebook's path for various loading/saving mechanisms
nb_path = os.getcwd()
td_path = os.path.dirname(os.path.realpath(nb_path))

In [2]:
import table_funcs as tbl_func
# Set a variable for current notebook's path for various loading/saving mechanisms
env_path = os.path.dirname(td_path)
env_path = td_path

In [3]:
import importlib
importlib.reload(cnss_func)

<module 'au_census_analysis_functions' from 'C:\\Users\\kempbri\\OneDrive - Mars Inc\\Documents\\GitHub\\CensusML\\app\\au_census_analysis_functions.py'>

In [4]:
full_category_list = ["Measure","Age", "Ancestry", "Birthplace", "Building occupation status", "Building type", 
"Country of Birth of Parents", "Dependent children in couple families", "Disability status",
"Education", "Educational status", "Family Composition", "Family number parents + children",
"Family type + Child age", "Family with children type", "Field of Study", "Heritage",
"Hours worked", "Household Composition", "Household type", "Income", "Indigenous", "Industry",
"Internet status", "Labour force status", "Labour Force Status of Female Parent",
"Labour Force Status of Male Parent", "Language", "Language Dialect", "Language of father", 
"Language of Parents", "Language of mother", "Language Spoken at Home", "Location", 
"Marital Status", "Mortgage", "Number", "Number of Commuting Methods", 
"Migration", "Number of Bedrooms", "Number of children", "Number of Hours", "Occupation", 
"Ownership", "Place of Birth", "Place of Residence class", "Place of Usual Residence", 
"Proficiency in Spoken English", "Relationship in Household", "Religion", "Religious Sect", 
"Rent", "Rental authority", "Rental cost", "School type", "Sex", "State", "Travel method type", 
"Unemployed Status", "Unpaid status", "Volunteering status", "Year of Arrival"]

full_tables_list = ["G01", "G02", "G03", "G04", "G05", "G06", "G07", "G08", "G09", "G10", 
"G11", "G12", "G13", "G14", "G15", "G16", "G17", "G18", "G19", "G20", "G21", "G22", "G23", 
"G24", "G25", "G26", "G27", "G28", "G29", "G30", "G31", "G32", "G33", "G34", "G35", "G36", 
"G37", "G38", "G39", "G40", "G41", "G42", "G43", "G44", "G45", "G46", "G47", "G48", "G49", 
"G50", "G51", "G52", "G53", "G54", "G55", "G56", "G57", "G58", "G59"]

In [5]:
# import Category_Measure_reference.csv as DF
df_cat_measure = pd.read_csv('{}\\Data\\Metadata\\Category_Measure_reference.csv'.format(env_path))

# filter measure column based on isin list
df_cat_measure = df_cat_measure[df_cat_measure['Category'].isin(full_category_list)]
df_cat_measure.head()

,Category,Measure,Appears_in_table
0,Age,Age_groups_0_4_years,G01
1,Age,Age_groups_5_14_years,G01
2,Age,Age_groups_15_19_years,G01
3,Age,Age_groups_20_24_years,G01
4,Age,Age_groups_25_34_years,G01


In [5]:
df_catf_working = pd.read_csv('{}\\Data\\Metadata\\Category_Measure_reference_temp.csv'.format(env_path))
df_catf_working.head()

,Measures,Categories
0,Total_Persons|Males,Total|Sex
1,Total_Persons|Females,Total|Sex
2,Total|Persons,Total|Total
3,Age_groups_0_4_years|Males,Age|Sex
4,Age_groups_0_4_years|Females,Age|Sex


In [6]:
cfw_fields = []
[cfw_fields.extend(y) for y in [x.split("|") for x in df_catf_working['Measures'].tolist()]]
len(cfw_fields)

49869

In [7]:
cfw_cats = []
[cfw_cats.extend(y) for y in [x.split("|") for x in df_catf_working['Categories'].tolist()]]
len(cfw_cats)

49869

In [8]:
dfcfw = pd.DataFrame()
dfcfw['Field'] = cfw_fields
dfcfw['Category'] = cfw_cats
len(dfcfw.drop_duplicates())

924

In [9]:
dfcfw.drop_duplicates().to_csv('{}\\Data\\Metadata\\Category_Measure_reference_n.csv'.format(env_path))

In [7]:
test_cat_lst = ['Aged_15_24_years|Age','Year_of_arrival_2012|Year of Arrival','Tenure_type_not_stated|Ownership','Age_25_34_years|Age']
[x.split("|") for x in test_cat_lst]

[['Aged_15_24_years', 'Age'],
 ['Year_of_arrival_2012', 'Year of Arrival'],
 ['Tenure_type_not_stated', 'Ownership'],
 ['Age_25_34_years', 'Age']]

In [8]:
test_dfx = pd.DataFrame()
test_dfx['Field'] = [i[0] for i in [x.split("|") for x in test_cat_lst]]
test_dfx['Cat'] = [i[1] for i in [x.split("|") for x in test_cat_lst]]
test_dfx.head()

,Field,Cat
0,Aged_15_24_years,Age
1,Year_of_arrival_2012,Year of Arrival
2,Tenure_type_not_stated,Ownership
3,Age_25_34_years,Age


In [30]:
test_tables_list = ['G01','G02','G03']

In [31]:
df_meta.head()

,Sequential,Short,Long,DataPack file,Profile table,Column heading description in profile,Table name,Table population,Age,Ancestry,...,Travel method type,Unemployed Status,Unpaid status,Volunteering status,Year of Arrival,Number of Classes,Number of Totals,Number of Classes Excl Total,Measures,Categories
0,G1,Tot_P_M,Total_Persons_Males,G01,G01a,Males,Selected Person Characteristics by Sex,Persons,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2,1,1,Total_Persons|Males,Total|Sex
1,G2,Tot_P_F,Total_Persons_Females,G01,G01a,Females,Selected Person Characteristics by Sex,Persons,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2,1,1,Total_Persons|Females,Total|Sex
2,G3,Tot_P_P,Total_Persons_Persons,G01,G01a,Persons,Selected Person Characteristics by Sex,Persons,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2,2,0,Total|Persons,Total|Total
3,G4,Age_0_4_yr_M,Age_groups_0_4_years_Males,G01,G01a,Males,Selected Person Characteristics by Sex,Persons,Age_groups_0_4_years,NaN,...,NaN,NaN,NaN,NaN,NaN,2,0,2,Age_groups_0_4_years|Males,Age|Sex
4,G5,Age_0_4_yr_F,Age_groups_0_4_years_Females,G01,G01a,Females,Selected Person Characteristics by Sex,Persons,Age_groups_0_4_years,NaN,...,NaN,NaN,NaN,NaN,NaN,2,0,2,Age_groups_0_4_years|Females,Age|Sex


In [32]:
table_rest_df = df_meta[df_meta['DataPack file'].str[:3].isin(test_tables_list)]
cats_in_tbls = table_rest_df[full_category_list].count()
cats_in_tbls = cats_in_tbls[cats_in_tbls>0]
cats_in_tbls.index.tolist()

['Measure',
 'Age',
 'Birthplace',
 'Education',
 'Indigenous',
 'Language',
 'Location',
 'Number',
 'Sex',
 'State']

In [5]:
all_categories = tbl_func.return_categories()
available_tables = tbl_func.return_tables()
available_measures = tbl_func.return_relevant_features()

## Filtering the datatable by category value
I want to be able to 

### Custom Functions
Below creates a number of custom functions. These are also replicated in the au_census_analysis_functions.py file but for the time being I am leaving them here if I need to re-run some of the cells executed prior to the implementation of these as imported functions.

In [3]:
# Show tables which have the above fields of information available
df_meta = pd.read_csv('{}\Data\Metadata\Metadata_2016_refined.csv'.format(td_path))
df_m_refined = df_meta[(df_meta['Age'] > 0) |
                      (df_meta['Income'] > 0) |
                      (df_meta['Occupation'] > 0) |
                      (df_meta['Family Composition'] > 0) |
                      (df_meta['Internet status'] > 0) |
                      (df_meta['Building occupation status'] > 0) |
                      (df_meta['Number of Bedrooms'] > 0) |
                      (df_meta['Place of Birth'] > 0) |
                      (df_meta['Country of Birth of Parents'] > 0) |
                      (df_meta['Year of Arrival'] > 0)]
tables_to_load = df_m_refined['Profile table'].unique().tolist()
load_features = ['Age','Income']

## Begin importing and exploring data

In [23]:
options_df = df_m_refined[['DataPack file','Table name']]
options_df = options_df.drop_duplicates(subset='DataPack file')
options_df = options_df.set_index('DataPack file')
options_df.head()

,Table name
DataPack file,
G01,Selected Person Characteristics by Sex
G03,Place of Usual Residence on Census Night by Age
G04A,Age by Sex
G04B,Age by Sex
G05,Registered Marital Status by Age by Sex


In [24]:
temp_df = pd.DataFrame(options_df['Table name'].value_counts()).reset_index()
temp_df.columns = ['Table name','Count']
options_df = options_df.reset_index().merge(temp_df, on='Table name')

In [27]:
options_df.loc[options_df.Count > 1, 'DataPack file'] = options_df['DataPack file'].str[:3]
options_df.drop_duplicates()

,DataPack file,Table name,Count
0,G01,Selected Person Characteristics by Sex,1
1,G03,Place of Usual Residence on Census Night by Age,1
2,G04,Age by Sex,2
4,G05,Registered Marital Status by Age by Sex,1
5,G06,Social Marital Status by Age by Sex,1
6,G07,Indigenous Status by Age by Sex,1
7,G09,Country of Birth of Person by Age by Sex,8
15,G10,Country of Birth of Person by Year of Arrival ...,3
18,G11,Proficiency in Spoken English/Language by Year...,4
22,G12,Proficiency in Spoken English/Language of Pare...,2


In [13]:
options_dict = [{'label': index, 'value': val} for index, val in options_df['Table name'].iteritems()]

options_dict

[{'label': 'G01', 'value': 'Selected Person Characteristics by Sex'},
 {'label': 'G03', 'value': 'Place of Usual Residence on Census Night by Age'},
 {'label': 'G04A', 'value': 'Age by Sex'},
 {'label': 'G04B', 'value': 'Age by Sex'},
 {'label': 'G05', 'value': 'Registered Marital Status by Age by Sex'},
 {'label': 'G06', 'value': 'Social Marital Status by Age by Sex'},
 {'label': 'G07', 'value': 'Indigenous Status by Age by Sex '},
 {'label': 'G09A', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09B', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09C', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09D', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09E', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09F', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09G', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09H', 'value': 'Country of Birth of Per

In [12]:
print ([x for x in options_df['Table name'].iteritems()])

[('G01', 'Selected Person Characteristics by Sex'), ('G03', 'Place of Usual Residence on Census Night by Age'), ('G04A', 'Age by Sex'), ('G04B', 'Age by Sex'), ('G05', 'Registered Marital Status by Age by Sex'), ('G06', 'Social Marital Status by Age by Sex'), ('G07', 'Indigenous Status by Age by Sex '), ('G09A', 'Country of Birth of Person by Age by Sex'), ('G09B', 'Country of Birth of Person by Age by Sex'), ('G09C', 'Country of Birth of Person by Age by Sex'), ('G09D', 'Country of Birth of Person by Age by Sex'), ('G09E', 'Country of Birth of Person by Age by Sex'), ('G09F', 'Country of Birth of Person by Age by Sex'), ('G09G', 'Country of Birth of Person by Age by Sex'), ('G09H', 'Country of Birth of Person by Age by Sex'), ('G10A', 'Country of Birth of Person by Year of Arrival in Australia'), ('G10B', 'Country of Birth of Person by Year of Arrival in Australia'), ('G10C', 'Country of Birth of Person by Year of Arrival in Australia'), ('G11A', 'Proficiency in Spoken English/Languag